In [3]:
from pymongo import MongoClient
import json
from glob import glob
from pprint import pprint

In [22]:
example = None
with open('../data/newsapi/newsapi_2024-05-05_06-55-50.json') as f:
    for line in f:
        example = json.loads(line)
        break

In [23]:
example

{'source': {'id': 'bbc-news', 'name': 'BBC News'},
 'author': 'https://www.facebook.com/bbcnews',
 'title': 'Politics no longer depends on the once-great UUP',
 'description': 'Budget issues send the UUP into opposition but the executive would carry on, writes Enda McClafferty.',
 'url': 'https://www.bbc.co.uk/news/uk-northern-ireland-68911089',
 'urlToImage': 'https://ichef.bbci.co.uk/news/1024/branded_news/DB6D/production/_133237165_afb7c9100952543e3b4bded30099c53db7aef03a.jpg',
 'publishedAt': '2024-04-27T07:00:24Z',
 'content': 'There was a time when big meetings of the Ulster Unionist Party were political box office, with good reason.\r\nDavid Trimble versus Jeffrey Donaldson - at stake the future of power sharing at Stormont… [+3578 chars]',
 '_id': '66372d50bc4d6ae9ad31aad0'}

In [ ]:
path_to_json = 'app/data/newsapi/*.json'
json_files = glob(path_to_json)

unprocessed = [file for file in json_files if 'processed' not in file]

In [ ]:
'''
{
    'source': 'El Pais',
    'url': 'http://www.elpais.com',
    'image': 'http://www.elpais.com',
    'title': 'El que sea',
    'description': 'El que sea',
    'author': 'El que sea',
    'date': 'El que sea',
    'content': 'El que sea',
    'api': 'El que sea',

}
'''

In [24]:
def check_if_exists(url, collection):
    if len(list(collection.find({'url': url}))) > 0:
        return True
    return False

def process_json(file, collection):
    data_mongo = []
    data_milvus = []
    with open(file) as f:
        for line in f:
            article = json.loads(line)
            if check_if_exists(article['url'], collection):
                continue
            data_mongo.append({
                'source': article['source']['name'],
                'url': article['url'],
                'image': article['urlToImage'],
                'title': article['title'],
                'description': article['description'],
                'author': article['author'],
                'date': article['publishedAt'],
                'content': article['content'],
                'api': 'newsapi'
            })
            data_milvus.append({
                'url': article['url'],
                'title': article['title'],
                'description': article['description']
            })
            
    return data_mongo, data_milvus

In [30]:
client = MongoClient('mongodb://localhost:27017/')
db = client['newsDB']
collection = db['articles']

with open('../data/newsapi/newsapi_2024-05-05_06-55-50.json') as f:
    for line in f:
        mon, mil = process_json('../data/newsapi/newsapi_2024-05-05_06-55-50.json')
        break

mil

[{'url': 'https://www.bbc.co.uk/news/uk-northern-ireland-68911089',
  'title': 'Politics no longer depends on the once-great UUP',
  'description': 'Budget issues send the UUP into opposition but the executive would carry on, writes Enda McClafferty.'},
 {'url': 'https://www.bbc.co.uk/news/world-europe-68887669',
  'title': "Spain's PM halts public duties as wife faces inquiry",
  'description': 'Pedro Sánchez says he will "stop and reflect" on whether to remain in the job.'},
 {'url': 'https://time.com/6969552/white-house-condemns-antisemitism-college-campuses-protests/',
  'title': 'White House Condemns Antisemitism on College Campuses',
  'description': 'The statement is the latest sign of how campus tensions spurred by the Israel-Hamas war are influencing U.S. politics in a presidential election year.'},
 {'url': 'https://www.wired.com/story/tiktok-ban-influencers-election/',
  'title': 'The TikTok Ultimatum Is Here. What Does It Mean?',
  'description': 'This week, we talk about t

In [ ]:
for file in unprocessed:
    print(file)
    with open(file) as f:
        for line in f:
            data = json.loads(line)
            print(data)
            break
    break